In [7]:
# import pandas and wbdata
import pandas as pd
import wbdata as wb

In [8]:
# Read the CSVs into dataframes
df_a = pd.read_csv('Resources/inpe_brazilian_amazon_fires_1999_2019.csv')
df_b = pd.read_csv('Resources/def_area_2004_2019.csv')

# Create a mapping dictionary
state_mapping = {
    'AC': 'ACRE',
    'AM': 'AMAZONAS',
    'AP': 'AMAPA',
    'MA': 'MARANHAO',
    'MT': 'MATO GROSSO',
    'PA': 'PARA',
    'RO': 'RONDONIA',
    'RR': 'RORAIMA',
    'TO': 'TOCANTINS'
}

# Map the full state names in df_a to their initials
df_a['state_initials'] = df_a['state'].map({v: k for k, v in state_mapping.items()})

# Convert the 'year' column in both dataframes to the same type
df_a['year'] = df_a['year'].astype(str)
df_b['Ano/Estados'] = df_b['Ano/Estados'].astype(str)

# Melt df_b to long format to facilitate the merge
df_b_melted = pd.melt(df_b, id_vars=['Ano/Estados'], value_vars=state_mapping.keys(), 
                      var_name='state_initials', value_name='deforestation_area')

# Merge the two dataframes on the year and state initials
merged_df = pd.merge(df_a, df_b_melted, left_on=['year', 'state_initials'], right_on=['Ano/Estados', 'state_initials'], how='outer')

# Filter to keep only data from 2004 onwards
merged_df = merged_df[merged_df['year'] >= '2004']

# Drop the state initials column and the duplicate 'Ano/Estados' column
merged_df.drop(['state_initials', 'Ano/Estados'], axis=1, inplace=True)

merged_df.head()


,year,month,state,latitude,longitude,firespots,deforestation_area
455,2004,1,AMAPA,0.131862,-51.642086,58,46.0
456,2004,2,AMAPA,0.686000,-50.987000,1,46.0
457,2004,4,AMAPA,-0.930000,-52.327000,1,46.0
458,2004,5,AMAPA,-0.919000,-52.286000,1,46.0
459,2004,6,AMAPA,1.473857,-51.059571,7,46.0


In [9]:
def filter_by_state_and_year(df, state, year):
    """
    Filters the dataframe for the specified state and year.

    Parameters:
    - df (DataFrame): The dataframe to filter.
    - state (str): The state to filter by.
    - year (str): The year to filter by.

    Returns:
    - DataFrame: The filtered dataframe.
    """
    return df[(df['state'] == state) & (df['year'] == year)]

# Usage
desired_state = "ACRE"
desired_year = "2004"
filtered_df = filter_by_state_and_year(merged_df, desired_state, desired_year)
filtered_df


,year,month,state,latitude,longitude,firespots,deforestation_area
549,2004,2,ACRE,-8.150750,-72.611250,4,728.0
550,2004,4,ACRE,-10.887444,-68.537000,9,728.0
551,2004,5,ACRE,-9.346143,-70.787952,21,728.0
552,2004,6,ACRE,-9.390833,-69.686500,12,728.0
553,2004,7,ACRE,-9.350074,-68.990400,95,728.0
554,2004,8,ACRE,-9.649285,-68.874877,1003,728.0
555,2004,9,ACRE,-9.835523,-68.803959,5458,728.0
556,2004,10,ACRE,-10.121528,-68.810804,633,728.0
557,2004,11,ACRE,-9.150760,-70.571200,25,728.0
558,2004,12,ACRE,-10.345182,-68.199364,11,728.0


In [18]:
def fetch_co2_emissions(country):
    """
    Fetches the CO2 emissions per capita data for the specified country for the years 2004 to 2019.

    Parameters:
    - country (str): The name of the country to fetch the data for.

    Returns:
    - List of tuples: The CO2 emissions per capita data for the specified country for the years 2004 to 2019.
    """
    # Set the indicator ID for CO2 emissions per capita
    indicator_id = "EN.ATM.CO2E.PC"

    # Fetch the data for the specified country
    data = wb.get_data(indicator_id, country=country)

    # Filter and store the data for the years 2004 to 2019 in a list of tuples
    co2_emissions_data = []
    for country_data in data:
        date = country_data['date']
        if 2004 <= int(date) <= 2019:
            country = country_data['country']['value']
            value = country_data.get('value', None)  # Use the get method to handle missing data
            try:
                co2_emissions_data.append((float(value), date, country))
            except:
                pass
    
    return co2_emissions_data


In [20]:
country_codes = ['BRA', 'CHN', 'IND', 'RUS', 'ZAF']
for country_code in country_codes:
    print(fetch_co2_emissions(country_code))
    

(2.05077012882977, '2019', 'Brazil')
(7.64543578560038, '2019', 'China')
(1.75253436625037, '2019', 'India')
(11.7971941673637, '2019', 'Russian Federation')
(7.68890762322173, '2019', 'South Africa')
